<a href="https://colab.research.google.com/github/wangscu/colab/blob/main/backtrader_grid_v002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

先经过统计学依据是均值回归后，再通过optuna找到最合适的网格交易参数。【零和游戏、仅为娱乐】

In [17]:
!pip install backtrader
!pip install yfinance
!pip install hurst
!pip install statsmodels
!pip install stochastic
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
%matplotlib notebook
import backtrader as bt
from datetime import datetime
from math import floor, ceil
import backtrader.analyzers as btanalyzers
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import optuna
import numpy as np
import statsmodels.api as sm
from hurst import compute_Hc

data = yf.download("BABA", '2022-01-01', '2023-05-26')
data.to_csv("baba.csv")

[*********************100%***********************]  1 of 1 completed


In [19]:
from statsmodels.tsa.stattools import adfuller
adfuller_test = adfuller(data['Close'])
adfuller_test

(-2.5084757560382345,
 0.11342567829188516,
 0,
 350,
 {'1%': -3.4491725955218655,
  '5%': -2.8698334971428574,
  '10%': -2.5711883591836733},
 1861.7039534298435)

In [20]:
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
data = pd.read_csv("baba.csv", index_col=0, parse_dates=True)
sel = ar_select_order(data['Open'], 10, old_names=False)
print(f"best lags:{sel.ar_lags}")
results = sel.model.fit()
print(results.summary())

best lags:[1]
                            AutoReg Model Results                             
Dep. Variable:                   Open   No. Observations:                  351
Model:                     AutoReg(1)   Log Likelihood                -967.645
Method:               Conditional MLE   S.D. of innovations              3.841
Date:                Sat, 03 Jun 2023   AIC                           1941.289
Time:                        01:45:56   BIC                           1952.863
Sample:                             1   HQIC                          1945.896
                                  351                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.0330      1.292      2.348      0.019       0.501       5.565
Open.L1        0.9675      0.013     73.342      0.000       0.942       0.993
                                    Ro

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [23]:
data = pd.read_csv("baba.csv", index_col=0, parse_dates=True)

hurst = compute_Hc(data['Close'], kind='price', simplified=True)
print(f"Hurst指数为：{hurst}")

# 计算半衰期
def half_life(data):
    lag = np.roll(data, 1)
    lag[0] = 0
    ret = data - lag
    ret[0] = 0
    lag2 = sm.add_constant(lag)
    model = sm.OLS(ret, lag2)
    res = model.fit()
    half_life = -np.log(2) / res.params[1]
    return half_life

half_life = half_life(data["Close"])
print("半衰期为：", half_life)

Hurst指数为：(0.4971994163518197, 1.5111920959106744, [[10, 17, 31, 56, 100, 177, 316, 351], [4.445691395771332, 5.714551904876474, 9.088112020540983, 11.930755634430659, 18.837792043113605, 15.911731184862218, 26.42103560326708, 27.285702257582603]])
半衰期为： 22.56687373108373


In [ ]:
# 计算Close的均值和标准差
mu = np.mean(data['Close'])
sigma = np.std(data['Close'])
print(f"mu:{mu}, simga:{sigma}")

mu:96.50301985998779, simga:15.589067979393205


In [ ]:
class GridStrategy(bt.Strategy):
  params = (('maxPrice', 110),('width', 4),('amount', 10),('maxGridHolding', 100),('baseHolding', 0),)

  def __init__(self):
    super().__init__()
    self.maxPrice = self.params.maxPrice
    self.width = self.params.width
    self.amount = self.params.amount
    self.maxGridHolding = self.params.maxGridHolding
    self.baseHolding = self.params.baseHolding

  def next(self):
    curentPrice = self.data[0]
    gridGap = (self.maxPrice-curentPrice)/self.width

    expectHoldingAfterBuying = min(max(floor(gridGap), 0) * self.amount, self.maxGridHolding) + self.baseHolding
    expectHoldingAfterSelling = max(ceil(gridGap), 0) * self.amount + self.baseHolding

    currentHolding = self.position.size

    if currentHolding < expectHoldingAfterBuying:
      if self.broker.get_cash() >= (expectHoldingAfterBuying-currentHolding) * curentPrice:
        self.buy(size=expectHoldingAfterBuying-currentHolding)
    elif currentHolding > expectHoldingAfterSelling:
      self.sell(size=currentHolding-expectHoldingAfterSelling)

  def stop(self):
    self.close()


In [ ]:
import optuna.logging as logging
#logging.disable_default_handler()

def objective(trial):
    maxPrice = trial.suggest_int('maxPrice', 80, 130)
    width = trial.suggest_int('width', 3, 7)
    amount = trial.suggest_int('amount', 5, 40)

    cash = 8000.0
    if maxPrice * width * amount * 4 > cash:
      return 0

    cerebro = bt.Cerebro()
    cerebro.broker.setcash(cash)
    cerebro.addstrategy(GridStrategy, maxPrice=maxPrice, width=width, amount=amount, maxGridHolding=1000)
    data = bt.feeds.PandasData(dataname=pd.read_csv("baba.csv", index_col=0, parse_dates=True))

    cerebro.adddata(data)
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpratio')
    cerebro.addanalyzer(btanalyzers.Returns, _name='returns')

    result = cerebro.run()

    sharpratio = result[0].analyzers.sharpratio.get_analysis()['sharperatio']
    return sharpratio


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000, n_jobs=-1)

print("Best trial:")
best_trial = study.best_trial
print("  Value: ", best_trial.value)
print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

import optuna.visualization as vis
vis.plot_optimization_history(study)    

[*********************100%***********************]  1 of 1 completed
Best trial:
  Value:  1.273615579540048
  Params: 
    maxPrice: 105
    width: 3
    amount: 6


In [ ]:
%matplotlib inline
from __future__ import (absolute_import, division, print_function, unicode_literals)

cerebro = bt.Cerebro()
cerebro.broker.setcash(8000.0)
cerebro.addstrategy(GridStrategy, maxPrice=105, width=3, amount=6, maxGridHolding=1000)
data = bt.feeds.PandasData(dataname=pd.read_csv("baba.csv", index_col=0, parse_dates=True))
cerebro.adddata(data)
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='sharpratio')
result = cerebro.run()

plt.rcParams['figure.figsize']=[15,12]
plt.rcParams.update({'font.size':12}) 
cerebro.plot(iplot=False)
plt.show()

NameError: ignored